I use the model [here](https://github.com/Pradhyo/machine-learning-practice-notebooks/blob/master/text-generation/top_lines.h5) from [this notebook](https://www.kaggle.com/pradhyo/keras-lstm-script-generator?scriptVersionId=9466911) to generate dialogs for each character of the office. While I did generate some random scripts in the other notebook, I will try to use better "seeds" to generate dialogs in this one.

I saved the model but forgot to save the word_indices dictionary without which the model is useless. Luckily, I can regenerate the dictionary again.

In [82]:
import keras
from keras.models import load_model
import numpy as np

In [83]:
# Get the script
def get_text():
    path = "the-office-all-episodes.txt"
    text = open(path).read().lower()
    return text

In [84]:
# Replace numbers with 0 - lots of permutations otherwise
def replace_numbers(text):
    for n in '0123456789':
        text = text.replace(n, "0")
    return text

In [85]:
from collections import Counter

text = get_text()
char_counts = Counter(text)
consider_words = ''.join(c for c in char_counts if not c.isalnum())
consider_words += '0'

In [86]:
def split_into_words(text, consider_words):
    # Split text into words - characters above are also considered words
    text = text.replace(' ', ' | ') # pick a char not in the above list
    text = text.replace('\n', ' | ') # pick a char not in the above list

    for char in consider_words:
        text = text.replace(char, f" {char} ") # to split on spaces to get char

    words_with_pipe = text.split()
    words = [word if word != '|' else ' ' for word in words_with_pipe]
    return words

In [87]:
model = load_model('top_lines.h5')
maxlen = 20

text = replace_numbers(text)
words = split_into_words(text, consider_words)
words_counter = Counter(words)

top_words = []
for word, count in words_counter.most_common(2000):
        top_words.append(word)

print(len(top_words))

def get_lines_with_words(top_words):
    selected_lines = []
    text = get_text()
    lines = text.split("\n")
    for line in lines:
        line = replace_numbers(line)
        words_in_line = split_into_words(line, consider_words)
        excluded_words = 0
        for word_in_line in words_in_line:
            if word_in_line not in top_words:
                excluded_words += 1
                break
        if not excluded_words:
            selected_lines.append(line)
    return selected_lines

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
                
                
selected_lines = get_lines_with_words(top_words)

selected_text = "\n".join(selected_lines)
selected_text = replace_numbers(selected_text)
selected_words = split_into_words(selected_text, consider_words)
unique_words = sorted(list(set(selected_words)))
word_indices = dict((word, unique_words.index(word)) for word in unique_words)

2000


In [108]:
# temperature between 0.2 to 1.2
def generate_dialog(seed, temperature=0.5):
    generated_text = seed
    # Don't print the entire seed, just the lasta two which are "actor:"
    temp_generated = ''.join(generated_text[-2:])
    for i in range(500):
        sampled = np.zeros((1, maxlen, len(unique_words)))
        for t, word in enumerate(generated_text):
            sampled[0, t, word_indices[word]] = 1.
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_word = unique_words[next_index]

        generated_text.append(next_word)
        generated_text = generated_text[1:]

        temp_generated+=(next_word) 
    print(temp_generated)
            
generate_dialog([" ", " ", " ", " ", " ", " ", " ", "what", " ", "are", " ", "you", " ", "doing", " ", "michael", "?", " ", "michael", ":"])

/Users/pbijja/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: RuntimeWarning: divide by zero encountered in log


michael: yeah, it's not a woman.   jim: yes, it's a damn. pam: no. i have a big deal know what it's a meeting.   pam: what are you doing? jim: [laughs] you just doing? jim: yeah, it's not a woman.   michael: [laughs] michael: [  phyllis: thanks. michael: i just don't know if i feel always like that for you and we can go of them.  michael: [laughs] michael: [- darryl: oh! jim: hey! erin: hey, you know maybe we were going to office a hug.    jim: how you doing? jim: yeah? darryl: how did you feel a lot of the day?   robert: so i'm going. michael: [on phone] hello. michael: hey, darryl. [  oscar if i'm not going to   angela: dunder mifflin. michael: okay. michael: oh.  dwight: [  stanley: [0: yes. michael: oh, my god! michael: maybe. michael: i'm sorry, i don't know.   stanley: [laughs]  michael: [on phone] michael god, we can't wait.   michael: thanks!   phyllis: hey. michael: hey, hey, hey, hey, hey. dwight: hey. you know what? i can't do it in there, he can right could do that.  darryl

In [102]:
# Get index of a random period "." in the existing script
def get_random_period_index():
    start_index = random.randint(maxlen, len(selected_words)-maxlen)
    
    c = ""
    while c != ".":
        c = selected_words[start_index]
        start_index += 1
    return (start_index - 1)

# Just add actor: next to 17 words before and including the period
def get_seed_for_actor(actor):
    period_index = get_random_period_index()
    seed = selected_words[period_index - 16 : period_index + 1] + [" ", actor, ":"]
    return seed


seed = get_seed_for_actor("michael")
print(seed)

[' ', 'the', ' ', 'most', ' ', 'important', ' ', 'christmas', ' ', 'party', ' ', 'of', ' ', 'my', ' ', 'life', '.', ' ', 'michael', ':']


In [103]:
# Get 30 most common actors
actors = Counter()
for _line in selected_lines:
    for _word in _line.split():
        if _word.endswith(":"):
            actors[_word] += 1
            
actors = (actors.most_common(30))
actors = [a[0][:-1] for a in actors]
print(actors)

['michael', 'jim', 'pam', 'dwight', 'andy', 'angela', 'kevin', 'erin', 'oscar', 'ryan', 'darryl', 'phyllis', 'jan', 'toby', 'kelly', 'stanley', 'holly', 'meredith', 'david', 'nellie', 'karen', 'creed', 'robert', 'roy', 'gabe', 'charles', 'pete', 'clark', 'deangelo', 'jo']


In [111]:
# print some dialogs for each actor
for actor in actors:
    print(f"\nActor: {actor}")
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f"Temperature: {temperature}")
        generate_dialog(get_seed_for_actor(actor), temperature)
        print("---\n")


Actor: michael
Temperature: 0.2


/Users/pbijja/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: RuntimeWarning: divide by zero encountered in log


michael: oh, no, no. [to dwight] [we be out up and no idea and the other of me, i feel like you know what? i mean, i just got it. i just want to know that i can go out of the guy.   phyllis: [on phone] hey, hey, you know if i say it in a hug.   jim: [on phone] yeah, i'm gonna say it now.  michael: [on phone] hey, hey, hey, hey, hey, hey, hey. michael: hey! michael: hey! michael: hey, hey, hey, hey, hey, hey, hey. good. i mean, i just wanted to have to do it with you? [laughs] michael: [laughs]  pam: [on phone] i can't tell you that for a boss. jim: yeah.   dwight: [on phone] hey, hey, darryl. [laughs] [we have to you and pam and pam's in my name. dwight: [0 out of the mall, god? [: scott? michael: oh, hey, hey. michael: hey, hey, hey, hey. i'm sorry? you know. i know. i know. i know. i really don't know. i know. i know. i know. i really don't want to go down. roy: hey. jim: hey! jim: hey, hey, hey, hey, hey
---

Temperature: 0.5
michael: yeah.   andy: it's okay. [laughs] kevin: i know.

pam: [on phone] yeah, i'm not gonna be in good. i mean, i know what up for a second.  michael: [laughs] [ve? kevin: [laughs] kevin: yeah, i can see you to go. dwight: i have to do.   andy: [  stanley: [dwight and roy] alright, i'm not sure we'll tell you. [laughs] roy: [  phyllis: but i can’ll go. jim: okay, i'm sorry but you can go up. michael: ok, good. i know, i know. i know, i know. i know. i really didn't get a time. pam: [laughs] pam: oh! jim: [on phone] [we have a :. michael: [laughs] i just didn't.   toby: yeah, it's not a lot of all out. michael: [on phone] hey, hey, hey, hey, how are you doing? jim: i can't tell you it to see it was so much. i'm here if i know. i know, i know. i know it's really funny. pam: i have a meeting.   michael: thanks. michael: yeah.    jim: yeah, yeah. i know it's not a woman.   pam: [on phone] hey. michael: hey, hey, hey, hey, hey. good.   stanley: we have a hug] you
---

Temperature: 1.0
pam: yes, i can do, uh... that was we don't tonight? michael:

andy: thanks you will take out from man. i'll stay and [um, stanley, kevin: oh. phyllis: yeah, we can you stay and stanley- andy stop it did, so much. and i can do that and i have to hear anyone like this in it? i want to have to do, uh, kelly. [ryan and at the other dog] [to andy] wait a used of anything. i know, um. so what are you doing? jan: and you keep them for this. pam: how do you feel really got it? phyllis: yes, i can't i just say it! pam: hi, how i have a woman? jan: yeah.    jan: pam's can go to you this name? pam: oh. [they have so of guy dunder ?:’s the m jim. pam: okay, kevin.   robert: wait a minute. what're going to get out out down. jan: michael. michael: yeah? michael: wait, you have to have to do this with me, it's really good. pam: alright, ok, can i talk to that is some manager? darryl: what do you say? kelly: ok.   creed: how will you go to the other people?  erin: we should get some thing together to know. pam: i know that's a good first room. erin: hey, um
---


...

karen: yes. pam: alright.   oscar: oh, i have a second. pam: oh, no, i'm not gonna work going to say it.   michael: i know.  dwight: [laughs] yes!  dwight: [  stanley: [dwight and oscar the office? [to michael] hey, you know, it's a big deal's gonna see you get a says that right know the first is a d. [  stanley: [0: hi guys? erin: [  stanley: [to dwight] look at her! michael: i can't do it on. i just want to go to the phone] jim: yeah. pam: yes, it sounds good. we're gonna get out of here.   andy: yeah. pam: uh-huh.  michael: i know. i know. i know. i'm sorry. i just want to do it? [michael and down and pam and pam and i know's jim and i know what up... did you have a second? michael: i mean, i can't tell you right now. it's what it was so much. [to michael] hey, hey, hey, darryl. [to jan] can we all tell you for a woman? it's not that i would be wrong. michael: how was it over you? pam: thank you. pam: yeah. michael: no, no, no. [looks 
---

Temperature: 1.0
karen: yeah. [ryan and an

robert: sure, um, i just got it. i think, he's gonna have to come back?  michael: yeah, yeah. i can see me? andy and andy.   darryl: oh, thank god! pam: yes! erin: who is this? michael: [0 up down to the room] dwight! toby: [laughs] dwight: oh, god.  michael: hey, you know, that's not here right now. i will here for my job. jim: yes. pam: [you hi." jim: [0: um! dwight: i know, i'm good. thank you. kevin: [to you] holly? michael: oh god! jim: all right! jim: alright. michael: yeah! ryan! david wallace: how do you know and a mistake to the desk? i'd be trying's someone and i'm is so much. jim: nice to meet you. [laughs] we're gonna do it get me in the meeting to make me go because i gotta be kidding.   stanley: yeah. pam: we have a idiot, she does what it was.  roy: what are you gonna get? what is off? dwight: just don't say? it's what it. pam: yeah are, i have a lot of the guy at my meeting to it. thanks. [to jim] are you kidding? he did you are i right with a wallace
---

Temperature: 

gabe: i'll find them. oscar: dwight!    phyllis: yes. toby: thanks. pam: [  toby: phyllis: is it?  jan: what it is? jan: come on.  david: [holly to michael scott. jan: hi, michael. michael: hey, hey. that's what that on your name. dwight: please. it was not a year did... come in get it.   ryan: i can go of this is so much for somebody a wants, but that was some time for a time, you know, i know? erin: and it was for your mistake for me. it's great. you don't want that later, god and that is a good idea, if i can go for a woman.  jim: [0 out to right? kelly: ok, how are you have to do? pam, will do. what? pam: oh! jim: yes. michael: oh, it's okay? stop. you - jim: okay. jim: alright, we're gonna know.  dwight: so i can do it for up! jan: can i help you? dwight: why don't here. michael: hey, we have not see... it really have a line!   phyllis: oh, stop. it’s out. and and like it.  michael: oh, um. [any] andy: okay. it's not. you can't, 
---


Actor: charles
Temperature: 0.2
charles: yeah

clark: hey, hey. i bet you know, i know.   andy: [  phyllis: [sir scott? jan: so we're gonna do it. michael: yeah.   michael: [on phone] hey, kevin. michael: hey, hey, hey, hey, hey. i'm sorry at all up. jim: alright, i mean it did jan like it's not a good thing. michael: i know it's not a woman.   andy: [  stanley: [on phone] hey, hey, hey, hey, hey, hey! jim: hey, hey, hey, hey, hey, hey, hey, hey. michael: hey! michael: hey! michael: hey, hey, hey, hey. michael: hey! michael: hey! michael: hey, hey, hey, hey, hey, hey. michael: hey! michael: hey! michael: hey, hey, hey, hey, hey, hey, hey, hey. michael: hey! michael: hey. michael: hey, hey, hey, hey, hey, hey, hey. michael: hey! jim: hey, hey. michael: hey! dwight: hey, hey, hey, hey. good. jim: hey, hey, hey. good. i didn't see you and then if you know and i can't do this in a good. jim: yeah.   pam: [on phone] hey, hey, how are you? jim: [  oscar and pam and pam like you didn't
---

Temperature: 0.5
clark: i didn't come in a woman

jo: give me hard. but darryl, why don’t you just leave you want to be on her? toby: i know, it was really nice of you god.   meredith: [right hi his 0? holly: and and i know, i know i'm going to go now. michael: thank you. michael: yeah, i know, it's not. michael: yeah.   pam: [on phone] hey, hey, hey, hey, hey, hey, hey! jim: hey! dwight: hey! michael: hey, hey, hey, hey, hey, hey, hey. michael: hey! how are you? pam: okay, ok, i know.   meredith: [on phone] yeah!   jim: yeah, yeah, i know. i know. i'm sorry, i can get it. i mean, but, i know. i know. it's good? dwight: how did you get up here? jim: [laughs]  pam: [on phone] hey, hey, it's how he you off.  jim: yes, we are here. pam: [  phyllis: [to stanley over michael] don't be okay. michael: thank you. michael: [  stanley: [0: hi guys? darryl: i just wanted to see you guys. i'm not the boss and i have to tell you it in the office for a second. jim: oh, yeah, uh, we can do
---

Temperature: 1.0
jo: well you don't do it? thanks. andy